In [ ]:
import scipy.io
import numpy as np
import pandas as pd

!pip install opencv-python
import cv2

import glob

import matplotlib.pyplot as plt
import matplotlib.image as mpimg


# Import classifiers
from sklearn import svm
from sklearn.svm import SVC

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
# metrics contain our plot_confustion_matrix and classification_report
from sklearn import metrics

# Helper fuction to splitting data
from sklearn.model_selection import train_test_split



In [ ]:
#result.to_csv(r'Facial_Expression_DF.csv', index = False)

In [ ]:
df = pd.read_csv('Facial_Expression_DF.csv')

classes = {1: 'suprise', 2: 'fear', 3: 'disgust' , 4: 'happiness', 5: 'sadness', 6: 'anger', 7 : 'neutral'}

df['expressions'] = df['expressions'].map(classes) #intger label to cat labes 

df = df.drop('file_name', axis = 1 )
df.head()

,expressions,0,1,2,3,4,5,6,7,8,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,sadness,71,101,74,66,82,55,84,50,41,...,173,174,174,173,173,172,173,175,177,177
1,sadness,11,11,37,1,12,27,32,35,44,...,31,33,17,17,18,24,10,13,18,23
2,happiness,9,13,14,14,22,32,30,20,12,...,215,215,215,215,215,215,213,213,213,212
3,happiness,50,33,17,20,36,50,50,45,38,...,224,222,219,215,213,212,213,211,207,204
4,sadness,23,20,21,29,39,43,38,30,23,...,52,64,79,103,165,208,227,160,159,177


In [ ]:
#check for nulls
#df.isnull().sum().sum() #0
#check for dups
df.duplicated().sum() #3
df.drop_duplicates()

,expressions,0,1,2,3,4,5,6,7,8,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,sadness,71,101,74,66,82,55,84,50,41,...,173,174,174,173,173,172,173,175,177,177
1,sadness,11,11,37,1,12,27,32,35,44,...,31,33,17,17,18,24,10,13,18,23
2,happiness,9,13,14,14,22,32,30,20,12,...,215,215,215,215,215,215,213,213,213,212
3,happiness,50,33,17,20,36,50,50,45,38,...,224,222,219,215,213,212,213,211,207,204
4,sadness,23,20,21,29,39,43,38,30,23,...,52,64,79,103,165,208,227,160,159,177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15334,neutral,145,93,103,116,94,90,106,129,132,...,73,85,80,56,32,20,34,43,51,47
15335,neutral,97,99,120,153,170,168,172,184,185,...,60,76,77,70,73,80,85,83,76,68
15336,neutral,49,47,49,54,61,65,63,60,65,...,38,40,42,42,40,37,35,37,39,42
15337,neutral,146,148,149,150,150,152,155,158,161,...,119,121,123,125,128,131,130,130,129,129


#### Expressions Classes 

surprise = 1, fear = 2 , disgust = 3 , happiness = 4 , sadness = 5 , anger = 6 and neutral = 7

In [ ]:
#class balance 
df.expressions.value_counts(), df.expressions.value_counts(normalize=True)

(happiness    5957
 neutral      3204
 sadness      2460
 suprise      1619
 disgust       877
 anger         867
 fear          355
 Name: expressions, dtype: int64,
 happiness    0.388356
 neutral      0.208879
 sadness      0.160376
 suprise      0.105548
 disgust      0.057175
 anger        0.056523
 fear         0.023144
 Name: expressions, dtype: float64)

In [ ]:
#taking out classes with that have counts less than 2.4k 

# if value_counts <= 2000 remove from df....
condition = ['happiness', 'sadness', 'neutral']

df2 = df[df.expressions.isin(condition)]
df2.expressions.value_counts(normalize=True)


happiness    0.512606
neutral      0.275708
sadness      0.211686
Name: expressions, dtype: float64

In [ ]:
sampled_happiness = df2[df2['expressions'] == 'happiness'].sample(n = 3000)
sadness =  df2[df2['expressions'] == 'sadness']

happy_sad = sampled_happiness.append(sadness, ignore_index = True )

In [ ]:
from sklearn.utils import shuffle

happy_sad= shuffle(happy_sad)

happy_sad.expressions.value_counts(normalize = True)


happiness    0.549451
sadness      0.450549
Name: expressions, dtype: float64

In [ ]:
 
selected_features = happy_sad.columns
 
selected_features = list(selected_features) # a list of all the cols

    #remove the file_name and the expressions col (our y value)
selected_features.remove('expressions')
    # selected_features #0-999 pixels of the image

condition_01 =  happy_sad['expressions'] == 'happiness'
happy_sad['expressions']  = np.where(condition_01, 1, 0)

x_cnn = happy_sad[selected_features]/255
y_cnn = happy_sad['expressions']

x_cnn.to_numpy()[0].reshape(-1, 100, 100, 1)

array([[[[0.87843137],
         [0.6745098 ],
         [0.35686275],
         ...,
         [0.65098039],
         [0.16470588],
         [0.75686275]],

        [[0.62352941],
         [0.44313725],
         [0.54117647],
         ...,
         [0.75686275],
         [0.24705882],
         [0.55294118]],

        [[0.37254902],
         [0.40392157],
         [0.61960784],
         ...,
         [0.82352941],
         [0.41568627],
         [0.38431373]],

        ...,

        [[0.84705882],
         [0.85490196],
         [0.86666667],
         ...,
         [0.86666667],
         [0.85882353],
         [0.83529412]],

        [[0.84313725],
         [0.85098039],
         [0.86666667],
         ...,
         [0.8627451 ],
         [0.85098039],
         [0.83137255]],

        [[0.83921569],
         [0.84705882],
         [0.8627451 ],
         ...,
         [0.85882353],
         [0.84705882],
         [0.82352941]]]])

In [ ]:
y_cnn

3030    0
4909    0
3432    0
273     1
281     1
       ..
4077    0
5121    0
578     1
3944    0
2245    1
Name: expressions, Length: 5460, dtype: int32

In [ ]:
# TensorFlow and tf.keras
#!pip install tensorflow
import tensorflow as tf
from tensorflow import keras
# Layers for our neural networks
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
# A pretrained model for transfer learning
from keras.models import Model
from keras.applications import vgg19

# Our normal python data science stack you've come to know and love
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import datasets

print(tf.__version__)

2.7.0


In [ ]:
cnn_df = happy_sad
len(cnn_df)
selected_features = cnn_df.columns
selected_features = list(selected_features) 

    
selected_features.remove('expressions')

    
cnn_x = cnn_df[selected_features]
cnn_y = cnn_df['expressions']
    
cnn_x_train, cnn_x_test, cnn_y_train, cnn_y_test = train_test_split(cnn_x, cnn_y, test_size=0.25, random_state= 43)    
    
# divide the values by 255 and reshape 
cnn_x_train = cnn_x_train / 255
cnn_x_test = cnn_x_test / 255

cnn_x_train = cnn_x_train.to_numpy()
cnn_x_test = cnn_x_test.to_numpy()
cnn_y_train = cnn_y_train.to_numpy()
cnn_y_test = cnn_y_test.to_numpy()

cnn_x_train = cnn_x_train.reshape(-1, 100, 100, 1)
cnn_y_train = cnn_y_train.reshape(-1)

cnn_x_test = cnn_x_test.reshape(-1, 100, 100, 1)
cnn_y_test = cnn_y_test.reshape(-1)  

In [ ]:
cnn_y_train

array(['happiness', 'happiness', 'sadness', ..., 'happiness', 'sadness',
       'happiness'], dtype=object)

In [ ]:



model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(input_shape=(100,100, 1),filters=100,kernel_size=(3,3), strides=(1,1), activation="relu"))
model.add(tf.keras.layers.Conv2D(filters=100,kernel_size=(3,3), strides=(1,1), activation="relu"))
model.add(tf.keras.layers.MaxPool2D(2, 2))

model.add(tf.keras.layers.Conv2D(filters=100,kernel_size=(3,3), strides=(1,1), activation="relu"))
model.add(tf.keras.layers.Conv2D(filters=100,kernel_size=(3,3), strides=(1,1), activation="relu"))
model.add(tf.keras.layers.MaxPool2D(2, 2))


model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(10000, activation='relu'))
model.add(tf.keras.layers.Dense(5000, activation='relu'))
model.add(tf.keras.layers.Dense(1000, activation='relu'))
model.add(tf.keras.layers.Dense(500, activation='relu'))
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(2, activation='softmax'))

In [ ]:
# compile your model

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['acc'])

In [ ]:
# fit your model

epochs = 6
model.fit(cnn_x_train, cnn_y_train, epochs=epochs, validation_split=0.2)


Epoch 1/6
103/103 [==============================] - 704s 7s/step - loss: 0.7190 - acc: 0.5382 - val_loss: 0.6834 - val_acc: 0.5763
Epoch 2/6
103/103 [==============================] - 692s 7s/step - loss: 0.6873 - acc: 0.5598 - val_loss: 0.6567 - val_acc: 0.6752
Epoch 3/6
103/103 [==============================] - 686s 7s/step - loss: 0.6053 - acc: 0.6905 - val_loss: 0.4952 - val_acc: 0.7680
Epoch 4/6
103/103 [==============================] - 676s 7s/step - loss: 0.4404 - acc: 0.8016 - val_loss: 0.3966 - val_acc: 0.8156
Epoch 5/6
103/103 [==============================] - 671s 7s/step - loss: 0.4274 - acc: 0.8178 - val_loss: 0.3771 - val_acc: 0.8425
Epoch 6/6
103/103 [==============================] - 668s 6s/step - loss: 0.3435 - acc: 0.8501 - val_loss: 0.3738 - val_acc: 0.8217


In [ ]:
# evaluate your results

model.evaluate(cnn_x_test, cnn_y_test)

43/43 [==============================] - 57s 1s/step - loss: 0.3796 - acc: 0.8425


[0.37955647706985474, 0.8424908518791199]

In [ ]:
import pickle 

pickle.dump(model, open('model.sav','wb'))

In [ ]:
amir_rocks = pickle.load(open('model.sav', 'rb'))
amir_rocks